In [ ]:
import argparse
import logging
import math
import os
import random

import datasets
from datasets import load_dataset, load_metric
from torch.utils.data.dataloader import DataLoader
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator
from transformers import (
    AdamW,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)
from transformers.utils.versions import require_version

# My custom model
from models import BertForSequenceClassification
from models import DeeBertForSequenceClassification
from models import BertConfig
import torch
import matplotlib.pyplot as plt

# Test my BertForSequenceClassification

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# change model pretrained path here
config = BertConfig(exit_port_threshold=0.1, entropy_threshold=0.2)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", config=config)

In [ ]:
inputs = tokenizer(["Hello, my dog is cute and I am the biggest person in the world"], 
                   max_length = 128, padding='max_length', truncation=True, return_tensors="pt")
labels = torch.tensor([1]).unsqueeze(0) # batch 1

In [ ]:
# first phrase
outputs = model(**inputs, labels=labels)

In [ ]:
# second phrase
outputs = model.exit_forward(**inputs, labels=labels)

In [ ]:
model.entropy_label[0]

In [ ]:
model.eval()

In [ ]:
inputs = tokenizer(["Hello, my dog is cute and I am the biggest person in the world Yo yo"], 
                   max_length = 128, padding='max_length', truncation=True, return_tensors="pt")
labels = torch.tensor([1]).unsqueeze(0)

In [ ]:
outputs = model.exit_inference_forward(**inputs)

In [ ]:
model.stop_layers

In [ ]:
outputs.logits

# Test DeeBert

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# change model pretrained path here
config = BertConfig(entropy_threshold=0.5)
model = DeeBertForSequenceClassification.from_pretrained("bert-base-uncased", config=config)

In [ ]:
inputs = tokenizer(["Hello, my dog is cute and I am the biggest person in the world haha, Excuse me"], 
                   max_length = 128, padding='max_length', truncation=True, return_tensors="pt")
labels = torch.tensor([0]).unsqueeze(0)

In [ ]:
# first phrase
outputs = model(**inputs, labels=labels)

In [ ]:
# second phrase
outputs = model.exit_inference_forward(**inputs, labels=labels)

In [ ]:
model.stop_layers

# Test result of new idea

In [ ]:
a = [torch.cat((torch.rand((32, 1)), torch.randint(0, 2, (32, 1))), dim = 1) for _ in range(0, 12)]

In [ ]:
X = a[0][:, 0].unsqueeze(1)
y = a[0][:, 1]

In [ ]:
X = torch.hstack((X, torch.zeros_like(X)))

In [ ]:
import matplotlib.pyplot as plt
from sklearn import svm

In [ ]:
clf = svm.SVC(kernel='linear')
clf.fit(X, y)

In [ ]:
w = clf.coef_[0]
x_0 = -clf.intercept_[0]/w[0]
margin = w[0]
w, x_0, margin

In [ ]:
plt.figure()
x_min, x_max = torch.floor(X.min()), torch.ceil(X.max())
y_min, y_max = -3, 3
yy = torch.linspace(y_min, y_max, 3)

In [ ]:
y[torch.where(y > 0)[0]]

In [ ]:
plt.scatter(X[torch.where(y > 0)[0]], torch.zeros_like(X[torch.where(y > 0)[0]]), c="red")
plt.scatter(X[torch.where(y < 1)[0]], torch.zeros_like(X[torch.where(y < 1)[0]]), c="blue")
plt.plot(x_0*torch.ones_like(yy), yy)

In [ ]:
optimal_point = []
fig, axs = plt.subplots(3, 4, figsize=(16, 9))
for i in range(0, 12):
    X = a[i][:, 0].unsqueeze(1)
    y = a[i][:, 1]
    X = torch.hstack((X, torch.zeros_like(X)))
    
    clf = svm.SVC(kernel='linear')
    clf.fit(X, y)
    optimal_point.append(-clf.intercept_[0]/w[0])
    
    x_min, x_max = torch.floor(X.min()), torch.ceil(X.max())
    y_min, y_max = -3, 3
    yy = torch.linspace(y_min, y_max, 3)
    
    axs[int(i/4), i%4].scatter(X[torch.where(y > 0)[0]], torch.zeros_like(X[torch.where(y > 0)[0]]), c="red")
    axs[int(i/4), i%4].scatter(X[torch.where(y < 1)[0]], torch.zeros_like(X[torch.where(y < 1)[0]]), c="blue")
    axs[int(i/4), i%4].plot(x_0*torch.ones_like(yy), yy)